## **Algorithmic Detection of Credit Card Defaulting**
#### **Final Project Phase 2**
*Tanvi Namjoshi, Dylan Van Bramer, Madeline Demers, Ella White*

In this report, we perform an initial examination of the dataset obtained from UC Irvine in 2016, focusing on credit card clients in Taiwan who have defaulted. Building a predictive model to assess the likelihood of customer defaulting requires fairness to prevent discrimination based on sensitive features. Defaulting on a credit card is defined as failing to make the minimum payment for at least 180 days. We plan to explore whether we can find a risk prediction model that is fair across different subgroups (male/female, education) that still remains accurate. 

The original source for the data can be found here: [https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients](https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients)

#### **Previous Work**
Our previous work involved auditing the Rotterdam welfare fraud detection algorithm using the synthetic dataset and model obtained by the data journalists at Lighthouse Reports and Wired. We spent the first half of our time working on Phase 2 using that dataset. This included translating the feature names from Dutch to English, calculating summary statistics for the dataset, and attempting to train an unsupervised model on the data provided. However, we decided to pivot from this to the credit card defaulting data for two main reasons. First, as noted in the methodology article from Lighthouse Reports, we do not have any of the target values about whether or not the individual did infact commit fraud. This made it extremely difficult for us to analyze the accuracy of the model used by the government in Rotterdam as well as our own model. Realistically, we would have only been able to use statisical parity as a metric for fairness, and we would have no ground truth labels. Secondly, the provided model was in R, and while we were able to load it into our notebook, the strucutre of the model was extremely unclear. We were unable to successfully use the Rotterdam model on the dataset. For these reasons we have pivoted to algorithmic detection of credit card defaulting.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#### **Data Import and Cleaning**

In [5]:
# Import the data
og_df = pd.read_csv('UCI_CC_defaulting.csv', skiprows=[0])

# Clean the data by dropping the additional first column, which is just an ID. 
# We can use index number as identification.  
og_df.drop(og_df.columns[0], axis=1, inplace=True)

# Display the Data
og_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
#In order to clean, we first check for missing values
missing_values = og_df.isnull().sum()
print(missing_values)

if missing_values.sum() == 0:
    print("No missing values found.")
else:
    print("There are missing values.")


LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64
No missing values found.


In [7]:
# Next, we check the data types. 
data_types = og_df.dtypes
print(data_types)

LIMIT_BAL                     int64
SEX                           int64
EDUCATION                     int64
MARRIAGE                      int64
AGE                           int64
PAY_0                         int64
PAY_2                         int64
PAY_3                         int64
PAY_4                         int64
PAY_5                         int64
PAY_6                         int64
BILL_AMT1                     int64
BILL_AMT2                     int64
BILL_AMT3                     int64
BILL_AMT4                     int64
BILL_AMT5                     int64
BILL_AMT6                     int64
PAY_AMT1                      int64
PAY_AMT2                      int64
PAY_AMT3                      int64
PAY_AMT4                      int64
PAY_AMT5                      int64
PAY_AMT6                      int64
default payment next month    int64
dtype: object


To clean our data further, we will re-format some of this information using one-hot encoding. From our data source, we know the dataset is structured as follows: 
* SEX: 1 = male, 2 = female 
* EDUCATION: 1 = graduate school, 2= university; 3 = high school; 4 = others
* MARRIAGE: 1 = married; 2 = single; 3 = others

In [88]:
# Create the one-hot encodings. For each feature, we first assert that all the info provided by the dataset creators is accurate. 
# After that, we create the one-hot encodings

clean_df = og_df
# SEX
if clean_df["SEX"].nunique() == 2 and np.array_equal(clean_df["SEX"].unique(), np.array([2,1])):
  print("resuts for SEX column match expectations")
else:
  # the above assertion failed. Let us remove all rows that do not match our expectations
  clean_df = clean_df[clean_df['SEX'].between(1,2)]
  assert (clean_df["SEX"].nunique() == 2) 
  print("removed SEX columns that did not mach expectations")


# EDUCATION
if clean_df["EDUCATION"].nunique() == 4:
  print("resuts for EDUCATION column match expectations")
else:
  # the above assertion failed. Let us remove all rows that do not match our expectations
  clean_df = clean_df[clean_df['EDUCATION'].between(1, 4)]
  assert (clean_df["EDUCATION"].nunique() == 4) 
  print("removed EDUCATION columns that did not mach expectations")


# MARRIAGE
if clean_df["MARRIAGE"].nunique() == 3:
  print("resuts for MARRIAGE column match expectations")
else:
  # the above assertion failed. Let us remove all rows that do not match our expectations
  clean_df = clean_df[clean_df['MARRIAGE'].between(1, 3)]
  assert (clean_df["MARRIAGE"].nunique() == 3) 
  print("removed MARRIAGE columns that did not mach expectations")

old_rows = og_df.count()[0]
new_rows = clean_df.count()[0]
print("The number of datapoints with bad values was ", old_rows - new_rows)

clean_df = pd.get_dummies(clean_df, columns=['SEX', 'EDUCATION', 'MARRIAGE' ], dtype=int)
column_names = {"SEX_1": "SEX_male", "SEX_2": "SEX_female", "EDUCATION_1": "graduate_education", "EDUCATION_2": "university_education", "EDUCATION_3": "highschool_education", "EDUCATION_4": "other_education"}
column_names.update({"MARRIAGE_1": "married", "MARRIAGE_2": "single", "MARRIAGE_3": "marriage_other"})
clean_df.rename(columns=column_names, errors="raise", inplace=True)
display(clean_df.head())

print("Finished creating one-hot encoding")



resuts for SEX column match expectations
removed EDUCATION columns that did not mach expectations
removed MARRIAGE columns that did not mach expectations
The number of datapoints with bad values was  399


,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,default payment next month,SEX_male,SEX_female,graduate_education,university_education,highschool_education,other_education,married,single,marriage_other
0,20000,24,2,2,-1,-1,-2,-2,3913,3102,...,1,0,1,0,1,0,0,1,0,0
1,120000,26,-1,2,0,0,0,2,2682,1725,...,1,0,1,0,1,0,0,0,1,0
2,90000,34,0,0,0,0,0,0,29239,14027,...,0,0,1,0,1,0,0,0,1,0
3,50000,37,0,0,0,0,0,0,46990,48233,...,0,0,1,0,1,0,0,1,0,0
4,50000,57,-1,0,-1,0,0,0,8617,5670,...,0,1,0,0,1,0,0,1,0,0


Finished creating one-hot encoding


In [21]:
#TODO: Do we also want to do past payment? The info is as follows:
#Past payment: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.

#### **Summary Statistics**


#### Part 2(a)

In [94]:
# Part 2 (a)
rows = clean_df.count()[0]
columns = len(clean_df.columns)
print("Looking at the whole dataset:")
print("There are", rows, "rows of data in the dataframe")
print("There are", columns, "features/columns in the dataframe")


#Per sensitive attribute subgroup

#Gender 
female_idx = clean_df.index[clean_df["SEX_female"]==1]
male_idx = clean_df.index[clean_df["SEX_male"] == 1]
print("\nLooking at the dataset by gender:")
print("The number of data points where the person's gender is female is: ", len(female_idx))
print("The number of data points where the person's gender is NOT female is: ", len(male_idx))

#Education 
graduate_idx = clean_df.index[clean_df["graduate_education"]==1]
university_idx = clean_df.index[clean_df["university_education"] == 1]
highschool_idx = clean_df.index[clean_df["highschool_education"] == 1]
other_education_idx = clean_df.index[clean_df["other_education"] == 1]

print("\nLooking at the dataset by education:")
print("The number of data points where the person has a graduate education is: ", len(graduate_idx))
print("The number of data points where the person has a univeristy education is: ", len(university_idx))
print("The number of data points where the person has a high school education is: ", len(highschool_idx))
print("The number of data points where the person has none of the above is: ", len(other_education_idx))


# Marriage Status
single_df = clean_df[clean_df['single'] == 1]
married_df = clean_df[clean_df['married'] == 1]
other_df = clean_df[clean_df['marriage_other'] == 1]
rows_married = married_df.shape[0]  
rows_single = single_df.shape[0]  
rows_other = other_df.shape[0]  

print("\nLooking at the dataset by marriage status:")
print("There are", rows_married, "datapoints where the person is married")
print("There are", rows_single, "datapoints where the person is single")
print("There are", rows_other, "datapoints where the person identifies as neither married or single")

#AGE 
under_fifty = clean_df.index[clean_df["AGE"]<50]
over_fifty = clean_df.index[clean_df["AGE"] >= 50]

print("\nLooking at the dataset by AGE:")
print("The number of data points where the person is under 50: ", len(under_fifty))
print("The number of data points where the person is over 50: ", len(over_fifty))




Looking at the whole dataset:
There are 29601 rows of data in the dataframe
There are 30 features/columns in the dataframe

Looking at the dataset by gender:
The number of data points where the person's gender is female is:  17855
The number of data points where the person's gender is NOT female is:  11746

Looking at the dataset by education:
The number of data points where the person has a graduate education is:  10581
The number of data points where the person has a univeristy education is:  14024
The number of data points where the person has a high school education is:  4873
The number of data points where the person has none of the above is:  123

Looking at the dataset by marriage status:
There are 13477 datapoints where the person is married
There are 15806 datapoints where the person is single
There are 318 datapoints where the person identifies as neither married or single

Looking at the dataset by AGE:
The number of data points where the person is under 50:  26967
The numbe

#### Part 2(b)
Our outcome variable of interest is whether the individual defaults on their payment next month. In our dataset that is the variable `default payment next month`, which is either 0 or 1.

In [108]:
outcome_variable = 'default payment next month'

mean_value = clean_df[outcome_variable].mean()
median_value = clean_df[outcome_variable].median()
std_dev_value = clean_df[outcome_variable].std()

# Print the results
print("Outcome variable:", outcome_variable)
print("\nOverall Statistics")
print("Mean:", mean_value)
print("Median:", median_value)
print("Standard Deviation:", std_dev_value)


Outcome variable: default payment next month

Overall Statistics
Mean: 0.2231343535691362
Median: 0.0
Standard Deviation: 0.4163547406844319


In [109]:

#Gender
print("\nStatistics by Gender")
mean_value = clean_df[outcome_variable][female_idx].mean()
median_value = clean_df[outcome_variable][female_idx].median()
std_dev_value = clean_df[outcome_variable][female_idx].std()
print("Mean for Females:", mean_value)
print("Median for Females :", median_value)
print("Standard Deviation for Females:", std_dev_value)

mean_value = clean_df[outcome_variable][male_idx].mean()
median_value = clean_df[outcome_variable][male_idx].median()
std_dev_value = clean_df[outcome_variable][male_idx].std()
print("Mean for Males:", mean_value)
print("Median for Males:", median_value)
print("Standard Deviation for Males:", std_dev_value)


Statistics by Gender
Mean for Females: 0.20968916269952395
Median for Females : 0.0
Standard Deviation for Females: 0.40709814501924513
Mean for Males: 0.24357227992508088
Median for Males: 0.0
Standard Deviation for Males: 0.4292557646242155


In [110]:
#Edcuation
print("\nStatistics by Education")
mean_value = clean_df[outcome_variable][graduate_idx].mean()
median_value = clean_df[outcome_variable][graduate_idx].median()
std_dev_value = clean_df[outcome_variable][graduate_idx].std()
print("Mean for People with Graduate Education:", mean_value)
print("Median for People with Graduate Education ", median_value)
print("Standard Deviation for Graduate Education:", std_dev_value)


mean_value = clean_df[outcome_variable][university_idx].mean()
median_value = clean_df[outcome_variable][university_idx].median()
std_dev_value = clean_df[outcome_variable][university_idx].std()
print("Mean for People with University Education:", mean_value)
print("Median for People with University Education:", median_value)
print("Standard Deviation for University Education:", std_dev_value)

mean_value = clean_df[outcome_variable][highschool_idx].mean()
median_value = clean_df[outcome_variable][highschool_idx].median()
std_dev_value = clean_df[outcome_variable][highschool_idx].std()
print("Mean for People with High School Education:", mean_value)
print("Median for People with High School Education ", median_value)
print("Standard Deviation for High School Education:", std_dev_value)


mean_value = clean_df[outcome_variable][other_education_idx].mean()
median_value = clean_df[outcome_variable][other_education_idx].median()
std_dev_value = clean_df[outcome_variable][other_education_idx].std()
print("Mean for People with Other Education:", mean_value)
print("Median for People with Other Education:", median_value)
print("Standard Deviation for Other Education:", std_dev_value)


Statistics by Education
Mean for People with Graduate Education: 0.19242037614592195
Median for People with Graduate Education  0.0
Standard Deviation for Graduate Education: 0.39422006872642057
Mean for People with University Education: 0.2373787792355961
Median for People with University Education: 0.0
Standard Deviation for University Education: 0.42549148512838436
Mean for People with High School Education: 0.25302688282372254
Median for People with High School Education  0.0
Standard Deviation for High School Education: 0.43479083865008655
Mean for People with Other Education: 0.056910569105691054
Median for People with Other Education: 0.0
Standard Deviation for Other Education: 0.23261919236784867


In [115]:
#Edcuation
print("\nStatistics by Marriage")
mean_value = married_df[outcome_variable].mean()
median_value = married_df[outcome_variable].median()
std_dev_value = married_df[outcome_variable].std()
print("Mean for married people:", mean_value)
print("Median for married people:", median_value)
print("Standard Deviation for married people:", std_dev_value)


mean_value = single_df[outcome_variable].mean()
median_value = single_df[outcome_variable].median()
std_dev_value = single_df[outcome_variable].std()
print("Mean for single people:", mean_value)
print("Median for single people:", median_value)
print("Standard Deviation for single people:", std_dev_value)

mean_value = other_df[outcome_variable].mean()
median_value = other_df[outcome_variable].median()
std_dev_value = other_df[outcome_variable].std()
print("Mean for People with Other Marriage Status:", mean_value)
print("Median for People with Other Marriage Status:", median_value)
print("Standard Deviation for Other Marriage:", std_dev_value)


Statistics by Marriage
Mean for married people: 0.2368479631965571
Median for married people: 0.0
Standard Deviation for married people: 0.425163989931608
Mean for single people: 0.2106162216879666
Median for single people: 0.0
Standard Deviation for single people: 0.40775917907231995
Mean for People with Other Marriage Status: 0.2641509433962264
Median for People with Other Marriage Status: 0.0
Standard Deviation for Other Marriage: 0.4415749014017929


In [116]:
print("\nStatistics by Age")

mean_value = clean_df[outcome_variable][under_fifty].mean()
median_value = clean_df[outcome_variable][under_fifty].median()
std_dev_value = clean_df[outcome_variable][under_fifty].std()
print("Mean for People Under 50:", mean_value)
print("Median for People Under 50 ", median_value)
print("Standard Deviation for People Under 50:", std_dev_value)

mean_value = clean_df[outcome_variable][over_fifty].mean()
median_value = clean_df[outcome_variable][over_fifty].median()
std_dev_value = clean_df[outcome_variable][over_fifty].std()
print("Mean for People Over 50:", mean_value)
print("Median for People Over 50 ", median_value)
print("Standard Deviation for People Over 50:", std_dev_value)





Statistics by Age
Mean for People Under 50: 0.21989839433381542
Median for People Under 50  0.0
Standard Deviation for People Under 50: 0.41418528699333007
Mean for People Over 50: 0.25626423690205014
Median for People Over 50  0.0
Standard Deviation for People Over 50: 0.4366523376693046


#### Part 2(c)
c. Plot at least as many figures as your # group members, and explain concisely but
meaningfully what the plot shows in markdown text

#### **Research Question, Hypotheses, and Analysis Plan**

a. Concretely, what is (are) your research question(s)? Be specific: what are the
inputs, outputs, and evaluation metrics you are interested in, and why?
b. What are your hypotheses?
i. E.g., do you notice any potential biases from your summary statistics?
What are they, and why might these exist?
c. What analyses are you going to run (in section 4) to test your hypotheses
presented above?

Research Question: Is it possible to fairly predict the risk of an indvidual defaulting on their credit card?
Hypotheses?: young, ummarried people that are college or lower level of education (irresponsible when younger?), take into account the fact that this is data from Tawian not USA
Evaluation Metrics: 
- calibrationl goal to improve this
- statisical parity; at least evaluate this

#### **Modeling**

a. You should run at least as many analyses as there are # group members.
i. E.g., if you have 3 group members, and your project is to find a risk
prediction model / method that balances fairness with accuracy, you may
want to try one logistic regression with [a subset of variables], one logistic
regression with [upsampling methods for minority groups in the training
data], and one random forest with [a different subset of variables].
ii. E.g., if you have 3 group members, and your project is to audit a dataset
for diversity, you could perform three separate analyses of the data – one
comparing the dataset makeup to the true underlying distribution by one sensitive attribute, a second similar comparison by a different sensitive
attribute, and an insertion of additional (collected or synthetic) data to
balance the dataset by sensitive attributes.
b. You should report evaluation metrics for each model, including by subgroup (for
your sensitive attribute(s) of interest).
i. If applicable, you should also include significance testing (e.g., report
p-values).
c. You should include informative plot(s) about your models.
i. E.g., you could plot ROC curves, or comparisons of fairness metrics
and/or performance across different models.


In [ ]:
df = pd.read_csv('UCI_CC_defaulting.csv')
df.head()
df = df.drop(df.columns[0], axis=1)
new_columns = df.iloc[0]
df = df.drop(0)
# Rename columns
df.columns = new_columns

df = df.reset_index(drop=True)

# Obtain our X and Y values
y = df['default payment next month']
y = [int(Y) for Y in y]
X = df.drop('default payment next month', axis=1)
d1 = X.copy()
d2 = X.copy()
d3 = X.copy()
d4 = X.copy()
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# We have one matrix of values for each of our data sets

In [ ]:
# Include all features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Don't evaluate on sensitive features
cols = ['EDUCATION', 'MARRIAGE', 'SEX', 'AGE']
d1 = d1.drop(cols, axis=1)
X_train1, X_test1, y_train1, y_test1 = train_test_split(d1, y, test_size=0.3)
# Don't consider marriage
d2 = d2.drop('MARRIAGE', axis=1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(d2, y, test_size=0.3)
display(d2)
# Don't consider age or education
cols = ['EDUCATION','AGE']
d3 = d3.drop(cols,axis=1)
X_train3, X_test3, y_train3, y_test3 = train_test_split(d3, y, test_size=0.3)
display(d3)

In [ ]:
# All Data
km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train)
preds = km.predict(X_test)
results = pd.DataFrame(X_test)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test))

km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train1)
preds = km.predict(X_test1)
results = pd.DataFrame(X_test1)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test1))

km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train2)
preds = km.predict(X_test2)
results = pd.DataFrame(X_test2)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test2))

km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train3)
preds = km.predict(X_test3)
results = pd.DataFrame(X_test3)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test3))


display(results)

#### **Results**

a. Interpret the results of each model.
i. Do your model estimates seem reasonable? (e.g., provide examples of
some model predictions like “for a 30 year old female patient, our model
predicts x% risk, whereas for a 30 year old male patient, our model
predicts y% risk”)
ii. What can you say about your model performance (e.g., based on the
evaluation metrics you calculated)?
iii. Do you have hypotheses for why your model performed well or not well?
b. Compare the performance of your models from part 4 on the evaluation metrics
you noted in section 3a.
i. What do you conclude?

#### **Contributions**

* Tanvi: 
* Ella: 
* Dylan: 
* Maddy: 

#### **Sources Cited**

Must include a citation of the data source(s) in ACM format.
b. Must include links to any sources used to assist with coding (do not need to be in
ACM format). Remember, any use of generative AI tools such as ChatGPT must
be cited including the query used, the answer output, and why you think the
answer is correct

Data Set: https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients